In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd 
import plotly.express as px
from get_similar_movies import similar_item_by_name

from user_utils import predict

In [2]:
# !pip install dash-ag-grid

In [3]:
movies=pd.read_csv("MovieLenDataset/movies.csv")
ratings=pd.read_csv("MovieLenDataset/ratings.csv")
tags=pd.read_csv("MovieLenDataset/tags.csv")
links=pd.read_csv("MovieLenDataset/links.csv")

#movie ids are the same in all datasets

## preprocess

In [4]:
watch_count = ratings.groupby(['movieId']).agg({'userId': 'count'}).reset_index()
watch_count.columns=["movieId","watch count"]
watch_count.head()

,movieId,watch count
0,1,215
1,2,110
2,3,52
3,4,7
4,5,49


In [5]:
avg_rating=ratings.groupby("movieId").agg({"rating":"mean"}).reset_index()
avg_rating.columns=["movieId","avg ratings"]

In [6]:
available_ratings=ratings["movieId"].unique().tolist()

In [7]:
movies=movies[movies["movieId"].isin(available_ratings)]


In [8]:
movies_merged=pd.merge(movies,links[["tmdbId","imdbId","movieId"]], on=["movieId"]).merge(watch_count[["movieId","watch count"]],on=["movieId"])
movies_merged.shape

(9724, 6)

In [9]:
movies_metadata=pd.merge(movies_merged,avg_rating[["movieId","avg ratings"]], on=["movieId"])

In [10]:
movies_metadata.head()

,movieId,title,genres,tmdbId,imdbId,watch count,avg ratings
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,862.0,114709,215,3.920930
1,2,Jumanji (1995),Adventure|Children|Fantasy,8844.0,113497,110,3.431818
2,3,Grumpier Old Men (1995),Comedy|Romance,15602.0,113228,52,3.259615
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,31357.0,114885,7,2.357143
4,5,Father of the Bride Part II (1995),Comedy,11862.0,113041,49,3.071429


----------------------------------------------------------------------------------------------------

# Dashboard 

In [11]:
# !pip install dash
# !pip install jupyter_dash

In [12]:
import dash
from dash import Dash, dcc, html,dash_table
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import dash_ag_grid as dag
import time

In [13]:
app = dash.Dash(__name__,external_stylesheets=["assets\quartz.min.css"])


In [14]:
#the tabs style when selected and unselected
tab_style = {
    "color": "white",
    'borderBottom': '1px solid #FCFCFC',
    'border-radius':'5px',
    "box-shadow": "1px 1px 0px F0A202",
    "border-left": "1px solid lightgrey !important",
   " border-right": "1px solid lightgrey !important",
    'border-radius':'5px',
    "border-top": "3px solid #e36209 !important",
    'background-image': 'linear-gradient(60deg, #B8336A, #947BD3,#49B6FF)',

    

}

tab_selected_style = {
    'fontWeight': 'bold',
   " box-shadow": "1px 1px 0px white",
    "border-left": "1px solid lightgrey !important",
   " border-right": "1px solid lightgrey !important",
    'border-radius':'5px',
    "border-top": "3px solid #e36209 !important",
    "background-color":"#FCFCFC"
    
    
}

In [15]:

tabs=dcc.Tabs(id='tabs-example-1', value='users', children=[
                            dcc.Tab(label='Users', value='users',style=tab_style, selected_style=tab_selected_style),
                            dcc.Tab(label='Movies', value='movies',style=tab_style, selected_style=tab_selected_style)
],content_style={"margin-top":"20px","margin-bottom":"-65px"},colors={"border-top": '#A4243B',})



In [16]:
dashboard_header=dbc.CardHeader(
    [html.H1("🎞Tailored Films🎞",style={"textAlign":"center","margin-top":"20px","margin-bottom":"20px",}),tabs], #"color":"#2E4057"
style={ "border-bottom-color": "#FCFCFC",'background-image': 'linear-gradient(-45deg,#0A2472,#49b6ff,#F0A7A0, #F26CA7)',
   
    })  #"background-color":'#FECF72'

In [17]:
content_card=dbc.Card([dashboard_header,dbc.CardBody([],id='tabs-example-content-1',style={"color":"#0D1B1E"})], style={"margin-top":"20px","background-color":"#FCFCFC","color":"#0D1B1E"})#give it id to return the button based on the mode  

In [18]:
#modals and paginantion
recommendation_modal = dbc.Modal(
    
    id="modal-1", fullscreen=True,  size="xl",is_open=False,style={"background-color":"white","color":"#0D1B1E"}
)


similarity_modal = dbc.Modal(
    id="modal-2", fullscreen=True,size="xl",is_open=False,style={"background-color":"white","color":"#0D1B1E"}
)

### Recommendation

In [19]:
# Create dropdown menu options

dropdown = dcc.Dropdown(id='user-dropdown', options=ratings["userId"].unique(), value=ratings['userId'].unique()[0],style={"margin-top":"20px","color":"#0D1B1E"})

button=dbc.Button("Get Recommendation for user", color="primary",size="lg",id="recommendation_button",style={"margin-top":"20px"}, className="btn btn-lg btn-primary")

# define the content for each modal as a dictionary




#add here input
user_content=dbc.Row([
    dropdown,dcc.Graph(id='graph-content',style={"margin-left":"40px","margin-top":"20px","color":"#0D1B1E"})]),\
    dbc.Row([dbc.Col([button,recommendation_modal],width=3) ],justify="center")
#add filteration for genres and ratings ??

In [20]:
@app.callback(
    Output('graph-content', 'figure'),
    Input('user-dropdown', 'value')
)
def update_graph(value):
    
    tagged_users=tags.userId.unique()
    grouped_tags=tags.groupby(["userId","movieId"]).agg({'tag': lambda x: ', '.join(x)}).reset_index()

    filtered_graph=ratings[ratings["userId"]==value]

    merged_df = pd.merge(filtered_graph, movies[['movieId', 'title',"genres"]], on='movieId').sort_values('rating', ascending=False)
    if value in tagged_users.tolist():
        merged_df2=pd.merge(merged_df,grouped_tags[["movieId","tag","userId"]],on="userId")
        return px.funnel(merged_df2, x='rating',y='title',color="rating",hover_data=["tags"],title=f"user's {value} interaction history",template="zephyr").update_layout(width=1000, height=600)
   
    else :
        return px.funnel(merged_df, x='rating',y='title',color="rating",hover_data=["genres"],title=f"user's {value} interaction history").update_layout(width=1000, height=600)


In [21]:
movies["title"]

0                                Toy Story (1995)
1                                  Jumanji (1995)
2                         Grumpier Old Men (1995)
3                        Waiting to Exhale (1995)
4              Father of the Bride Part II (1995)
                          ...                    
9737    Black Butler: Book of the Atlantic (2017)
9738                 No Game No Life: Zero (2017)
9739                                 Flint (2017)
9740          Bungo Stray Dogs: Dead Apple (2018)
9741          Andrew Dice Clay: Dice Rules (1991)
Name: title, Length: 9724, dtype: object

In [22]:
movie_dropdown = dcc.Dropdown(id='movie-dropdown', options=movies["title"].unique(), value=movies["title"][0],style={"margin-top":"20px","margin-bottom":"20px","color":"#0D1B1E"})
button=dbc.Button("Get Similar movies", color="primary",size="lg",id="similarity_button",style={"margin-top":"20px","margin-bottom":"20px"}, className="btn btn-lg btn-primary") #color="primary" return to dbc size="md"


#add here input too
movies_content=dbc.Row([movie_dropdown, html.Div(id='table-container')]),dbc.Row([dbc.Col([button,similarity_modal],width=3) ],justify="center")

In [23]:
@app.callback(
    Output('table-container', 'children'),
    Input('movie-dropdown', 'value')
)
def update_table(value):
    movies_filtered=movies_metadata[movies_metadata["title"]==value ]
    
    

    table = dbc.Table.from_dataframe(
    movies_filtered,
    bordered=True,
    hover=True,
   
    color="table-secondary"
#     className="table-light",
    
    
  )
    
    
    return table
    

In [24]:
@app.callback(
    Output('tabs-example-content-1', 'children'),
    Input('tabs-example-1', 'value')
)
def render_content(tab):
    if tab == 'users':
        return user_content
    elif tab == 'movies':
        return movies_content


In [32]:
# open modal - user's modal
@app.callback(
    Output("modal-1", "is_open"),
    Input("recommendation_button", "n_clicks"),
    State("modal-1", "is_open"),
)
def toggle_modal_1(n_clicks, is_open):
    if n_clicks:
        return True
    else:
        return False

   

    ##movies callback
@app.callback(
    Output("modal-2", "is_open"),
    Input("similarity_button", "n_clicks"),
    State("modal-2", "is_open"),
)
def toggle_modal_2(n_clicks, is_open):
    if n_clicks:
        return True
    else:
        return False
    
    
#----------------------------------------------------------------------------------------

# update recommendation modal's content for user
@app.callback(
  Output("modal-1", "children"),
  Input('user-dropdown', 'value')
)
def update_modal_content(value):
    ##add model function 
    ##add daq table to be viewed with dcc.loading
  
    columnDefs = [
    {"headerName": "Movie ID","field": "movieId", 'headerStyle': {'background-image': 'linear-gradient(60deg, #C2E7FF, #CDC2EB,#F2CFDD)'}},
    {"headerName": "Title","field": "title"},
    {"headerName": "Genres","field": "genres"},
    {"headerName": "TMDB ID","field": "tmdbId"},
    {"headerName": "IMDB ID","field": "imdbId"},
    {"headerName": "Watched count","field": "watch count"},
    {"headerName": "Average Rating","field": "avg ratings"},
    {"headerName": "Predicted Rates","field": "rates"},
]
    
    input_field=dcc.Input(id="input-page-size", type="number", min=1, max=len(movies_metadata), value=5)
    
    #get prediction
    user_df=predict(value)
    filtered=movies_metadata[movies_metadata["movieId"].isin(user_df.movieId.unique().tolist())]
    merged=pd.merge(filtered,user_df, on=["movieId"])
    user_recommendations=merged.sort_values(by='rates', ascending=False)
    
    table=dag.AgGrid(
            id="grid-page-size",
            columnDefs=columnDefs,
        
            #where to change
            rowData=user_recommendations.to_dict("records"),
            columnSize="sizeToFit",
            defaultColDef={"resizable": True,"suppressMovable": True},
            dashGridOptions={"pagination": True},
            suppressDragLeaveHidesColumns=True,
            className="ag-theme-material headers1 borders",
            rowStyle={'background-image': 'linear-gradient(60deg, #C2E7FF, #CDC2EB,#F2CFDD)'},
        
        )
    
    #can be removed
    loading_table = dcc.Loading(
    type="dot",
    id="loading-table",
    children=[table]
)

    
    modal_1_content =\
    dbc.ModalHeader([dbc.ModalTitle(f"Movie Recommendation for user's {value}")],style={'textAlign':'center',"color":"white",'background-image': 'linear-gradient(60deg, #B8336A, #947BD3,#49B6FF)'}),\
    dbc.ModalBody([
        dbc.Row([ 
            dbc.Col([html.P("Please enter number of recommendations")],width=4),dbc.Col([input_field],width=3,style={"margin-left":"-35px", 'fontWeight': 'bold'})]),
        loading_table,
        
       
        ],id="recommendation_modal",) #style={'background-image': 'linear-gradient(60deg, #B8336A, #947BD3,#49B6FF)'}

    return modal_1_content



In [33]:
#update number of columns
@app.callback(
    Output("grid-page-size", "dashGridOptions"),
    Input("input-page-size", "value"),
    State("grid-page-size", "dashGridOptions"),
)
def update_page_size(page_size, grid_options):
    page_size = 1 if page_size is None else page_size
    grid_options["paginationPageSize"] = page_size
    return grid_options

In [34]:
#update similarity's modal's content
@app.callback(
  Output("modal-2", "children"),
  Input('movie-dropdown', 'value')
)
def update_modal_content2(value):
    ##add model function 
    ##add daq table to be viewed with dcc.loading
  
   
    columnDefs = [
    {"headerName": "Movie ID","field": "movieId"},
    {"headerName": "Title","field": "title"},
    {"headerName": "Genres","field": "genres"},
    {"headerName": "TMDB ID","field": "tmdbId"},
    {"headerName": "IMDB ID","field": "imdbId"},
    {"headerName": "Similarity score","field": "similarity score"},
    {"headerName": "Average Rating","field": "avg ratings"},
    {"headerName": "Watched count","field": "watch count"},
]
    
    input_field=dcc.Input(id="input-movie-size", type="number", min=1, max=len(movies_metadata), value=5)
    similar_films=similar_item_by_name(value)
    #add hovering effect on titles
    table=dag.AgGrid(
            id="grid-movie-size",
            columnDefs=columnDefs,
            #where to change
            rowData=similar_films.to_dict("records"),
            columnSize="sizeToFit",
            defaultColDef={"resizable": True,"suppressMovable": True},
            dashGridOptions={"pagination": True},
            suppressDragLeaveHidesColumns=True,
            className="ag-theme-material headers1 borders",
            rowStyle={'background-image': 'linear-gradient(60deg, #C2E7FF, #CDC2EB,#F2CFDD)'},

        
        
            
        )
    
    loading_table = dcc.Loading(
    type="dot",
    id="loading-items",
        
    children=[table]
)

    
    modal_2_content =\
    dbc.ModalHeader(dbc.ModalTitle(f"Similar movies for for: {value}"),style={'textAlign':'center',"color":"white",'background-image': 'linear-gradient(60deg, #B8336A, #947BD3,#49B6FF)'}),dbc.ModalBody([
        dbc.Row([ 
        dbc.Col([html.P("Please enter number of recommendations")],width=4),dbc.Col([input_field],width=3,style={"margin-left":"-35px"})
        ]),
        loading_table,
       
    ],id="similarity_modal")

    return modal_2_content



In [35]:
#update number of columns
@app.callback(
    Output("grid-movie-size", "dashGridOptions"),
    Input("input-movie-size", "value"),
    State("grid-movie-size", "dashGridOptions"),
)
def update_page_size2(page_size, grid_options):
    page_size = 1 if page_size is None else page_size
    grid_options["paginationPageSize"] = page_size
    return grid_options

In [36]:
footer = html.Footer(
       
            html.P("🤞 ITI © Salma Kishk 💟 Mariam Gamal 💟 Yasmeen Nour 🤞",style={'textAlign':'center',"margin-top":"50px","font-color":"#808080"})
)
    


In [37]:
app.layout = dbc.Container([
                            content_card,
                            footer
                          ])

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False,port=8050)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [25/Jun/2023 22:26:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2023 22:26:28] "GET /assets/_bootswatch.scss?m=1686863302.6586335 HTTP/1.1" 304 -
127.0.0.1 - - [25/Jun/2023 22:26:28] "GET /assets/quartz.min.css HTTP/1.1" 304 -
127.0.0.1 - - [25/Jun/2023 22:26:28] "GET /assets/quartz.min.css?m=1687717675.3518233 HTTP/1.1" 304 -
127.0.0.1 - - [25/Jun/2023 22:26:28] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2023 22:26:28] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2023 22:26:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2023 22:26:29] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Jun/2023 22:26:29] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Jun/2023 22:26:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2023 22:26:30] "GET /_dash-component-suites